In [2]:
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

dico_annee = {"2023_2024" : ["Auxerre", "Angers", "Saint-Étienne", "Rodez", "Paris FC"],
              "2022_2023" : ["Le Havre", "Metz", "Bordeaux", "Bastia", "Caen"],
              "2021_2022" : ["Toulouse", "AC Ajaccio", "Auxerre", "Paris FC", "Sochaux"],
              "2020_2021" : ["Troyes", "Clermont Foot", "Toulouse", "Grenoble Foot", "Paris FC"]}

In [3]:
annee = list(dico_annee.keys())[0]

In [4]:
event_import = pd.read_json(f"../../Data_file/Heatmap SB/{annee}.json")

In [8]:
event = event_import[~event_import.location.isna()]

event.sort_values(by = ["match_id", "index"], inplace = True)

event.index = range(len(event))

centre = event[(event.pass_cross == 1) & (event.pass_type != "Corner")]

dico_event_test = {"Nb_but" : {}, "Event avant but" : {}}
for n_event in range (3, 9) :
    liste_event = []
    count_goal = 0
    for i in centre.index :
        team_i = centre.team
        event_i = event.loc[i + 1 : i + n_event]
        if sum(((event_i.shot_outcome == "Goal") | (event_i.type == "Own Goal Against")) & (event_i.team == centre.loc[i, "team"])) > 0 :
            loc_goal = event_i[((event_i.shot_outcome == "Goal") | (event_i.type == "Own Goal Against")) & (event_i.team == centre.loc[i, "team"])].index.values
            liste_event.append(event.loc[i + 1 : loc_goal[0], "type"].tolist())
            count_goal += 1
        dico_event_test["Nb_but"][n_event] = count_goal
        dico_event_test["Event avant but"][n_event] = liste_event

In [9]:
df_event_test = pd.DataFrame(index = range(3, 9))
for key in dico_event_test["Event avant but"].keys() :
    for i in dico_event_test["Event avant but"][key] :
        for j in i :
            if j not in df_event_test.columns :
                df_event_test = pd.concat([df_event_test, pd.Series(0, index = df_event_test.index, name = j)], axis = 1)
            df_event_test.loc[key, j] += 1

In [10]:
pd.concat([pd.Series(dico_event_test["Nb_but"], name = "Nombre but"), df_event_test], axis = 1)

,Nombre but,Ball Receipt*,Duel,Shot,Carry,Pressure,Error,Block,Ball Recovery,Goal Keeper,Clearance,Foul Committed,Pass,Miscontrol,Dribbled Past,Dribble,Foul Won
3,209,204,21,209,10,34,1,2,3,2,1,1,0,0,0,0,0
4,227,222,23,227,16,41,1,7,10,2,7,2,2,0,0,0,0
5,257,261,25,262,29,51,5,10,25,10,17,2,12,1,0,0,0
6,278,290,30,288,37,66,6,14,35,17,21,5,20,2,1,1,3
7,291,309,31,308,43,75,7,18,41,25,26,7,29,2,1,1,4
8,307,338,38,332,53,86,8,21,51,34,30,8,44,2,2,3,5


In [48]:
pd.concat([pd.Series(dico_event_test["Nb_but"], name = "Nombre but"), df_event_test], axis = 1).mean(axis = 0)

Nombre but        242.75
Ball Receipt*     244.25
Duel               24.75
Shot              246.50
Carry              23.00
Pressure           48.00
Error               3.25
Block               8.25
Ball Recovery      18.25
Goal Keeper         7.75
Clearance          11.50
Foul Committed      2.50
Pass                8.50
Miscontrol          0.75
Dribbled Past       0.25
Dribble             0.25
Foul Won            0.75
dtype: float64